# 「文字列情報のone-hotエンコーディング」と「検証用データを用いたモデルの評価」
前回は以下のことを学びました．
 - pandasによるcsvの読み込みと簡単な操作（列・行へのアクセス，`numpy.ndarray`への変換）
 - sklearnの基本的な使い方
 - 提出までの流れ：
   1. 特徴ベクトルの作成
   2. モデル・アルゴリズムの選定，学習
   3. 予測
   4. 提出

その一方で，以下のことは**行いませんでした**：
 - 文字列情報の利用：文字列で表現されている"type"や"region"，"Date"の情報はどうやって用いるのか？
 - 予測の投稿前の定量的な評価：バイアス項を用いない線型回帰`LinearRegression(fit_intercept=False)`は悪そうだが，本当に悪いのか？

## 今回学ぶこと
 - 文字列情報のone-hotエンコーディング
 - 検証用データを用いたモデルの投稿前の評価


## （復習）予測モデル構築の流れ

機械学習を用いて予測モデルを構築し，未知の（テスト）データに対して予測を行う手順は，主に以下のようになります．
1. データを用意し，特徴ベクトルを作る
2. どのような手法（モデル）を使うかを決める
3. モデルを学習する方法を決め，学習する
4. 未知のデータに予測を行う（そして本演習では提出する）


## （復習）データ読み込み
前回と同様に，まずはデータの読み込みを行います．
データの読み込みはpandasの`read_csv`で行えます．
ライブラリを使うためには`import`する必要があります．
詳細は前回の資料を参照してください．

In [1]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression 

In [ ]:
Colabの場合は次のセルを（コメントアウトを外して）動かし，更にその次のパスを適宜変更してください．

In [2]:
from google.colab import drive
drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）

Mounted at /content/drive


In [3]:
# 動的型付け言語なので，変数の型の宣言は不要
# Google Colabの場合
d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった
d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む． TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった
# Jupyter Note の場合
#d_train = pd.read_csv("data/train.csv") # 訓練データを読み込む
#d_test = pd.read_csv("data/test.csv") # テストデータを読み込む


前回と同じように，読み込んだデータを表示して確認します．

In [4]:
print("訓練データ")
print(d_train)
print("\nテストデータ")
print(d_test)

訓練データ
       Unnamed: 0        Date  AveragePrice        4046        4225  \
0              32  2016-05-15          0.82  1134719.15   643464.88   
1              43  2016-02-28          0.89  1873878.11  2020327.87   
2              15  2017-09-17          2.94      181.23      861.34   
3              10  2018-01-14          0.81   628591.78   284087.44   
4              14  2017-09-24          1.57        1.21      105.02   
...           ...         ...           ...         ...         ...   
12769          47  2015-02-01          1.44     1565.49     4210.72   
12770          46  2016-02-07          1.07     7720.21    22562.00   
12771          43  2015-03-01          1.92      386.65     5898.07   
12772           9  2015-10-25          1.86      441.78     1817.74   
12773          28  2015-06-14          0.92  3239258.39  2416002.43   

            4770  Small Bags  Large Bags  XLarge Bags          type  \
0       95527.46   871562.92   104146.00     17435.17  conventional   

前回と同様にいくつかの変数を用意します．
 - `n_train`, `n_test`：訓練データ数，テストデータ数.
 - `y_train`：訓練データの目標値．`pop`メソッドによって元のデータフレームから取り除いて作る．

In [5]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")
print("\n 出力情報を取り出す．AveragePriceの列が消えている")
# targetの値
y_train = d_train.pop('AveragePrice')
y_train = y_train.values # numpyのarrayに変換
print(d_train)
print(y_train)

訓練データとテストデータの数を取得
訓練データ数：12774，テストデータ数：5475

 出力情報を取り出す．AveragePriceの列が消えている
       Unnamed: 0        Date        4046        4225       4770  Small Bags  \
0              32  2016-05-15  1134719.15   643464.88   95527.46   871562.92   
1              43  2016-02-28  1873878.11  2020327.87  302210.56  2091747.51   
2              15  2017-09-17      181.23      861.34       0.00      312.22   
3              10  2018-01-14   628591.78   284087.44   15249.83   200603.91   
4              14  2017-09-24        1.21      105.02       0.00    16555.36   
...           ...         ...         ...         ...        ...         ...   
12769          47  2015-02-01     1565.49     4210.72       0.00     1133.33   
12770          46  2016-02-07     7720.21    22562.00      37.04    38739.27   
12771          43  2015-03-01      386.65     5898.07       0.00      516.67   
12772           9  2015-10-25      441.78     1817.74      15.61       72.11   
12773          28  2015-06-14  3239258.39  

ここまで動かしてきたセルは今後の回でも毎回のように動かします．次回以降は説明を省略します．

## （復習）数値情報だけの特徴ベクトルを作る

またしても復習ですが，**まず**前回と同じ特徴ベクトルを作ります．
入力の情報として，以下が与えられています：
 - Date：日付
 - 4046, 4225, 4770：小，大，特大として売れた数
 - Small Bags, Large Bags, XLarge Bags：小，大，特大の袋として売れた数
 - type：オーガニックか否か
 - region：地域

この中で，"Date"，"type"，"region"が文字列（string)の情報です．
前回はこれらを用いずに，"4046"，"4225"，"4770"，"Small Bags"，"Large Bags"，"XLarge Bags"の**元から数値的な情報である6つだけ**を用いました．
pandasではブラケット（角括弧）`[]`を用いることで特定の列や行を取り出すことができ，また`.values`で配列の中身だけを取り出せるのでした．
したがって，以下のようにすることで数値の情報だけを用いた特徴ベクトルの行列を作ることができます．

In [6]:
columns_num = ["4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags"]
X_train_num = d_train[columns_num].values
X_test_num = d_test[columns_num].values

## （復習）線形回帰の学習と予測
sklearnを用いる基本的な手順は，
1. モデルのインスタンスを作成
2. 作成したモデルオブジェクトを**`fit`メソッド**を用いて学習．`fit`メソッドには訓練データの入力と目標値（つまり，行列とベクトル）を渡す．
3. 学習したモデルを用いて**`predict`メソッド**で予測．`predict`メソッドにはデータの入力（つまり行列）を渡す．

でした．以下のようにして実行できます．

In [7]:
# 手順1：LinearRegressionのインスタンスの作成
lr = LinearRegression()
# 手順2：上で作ったオブジェクトの学習
lr.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test_lr = lr.predict(X_test_num)
print(y_pred_test_lr)

[1.42066402 1.42219244 1.39648389 ... 1.42335768 1.41967346 1.4463927 ]


ついでに，**バイアス項を使わない**線形回帰も学習しておきます．
インスタンスを作成する時に，`fit_intercept=False`とすればよいのでした．

In [8]:
# 手順1：LinearRegressionのインスタンスの作成．fit_intercept=Falseとすることで，バイアス項を使わない
lr_without_bias = LinearRegression(fit_intercept=False)
# 手順2：上で作ったオブジェクトの学習
lr_without_bias.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test_lr_without_bias = lr_without_bias.predict(X_test_num)
print(y_pred_test_lr_without_bias)

[ 0.00021382  0.03226085  0.16998403 ...  0.03687215 -0.0010252
  0.10894106]


これで最低限の準備は終了です．

## 文字列情報のone-hotエンコーディング

文字列情報である"Date"，"type"，"region"は，値段の予測に役に立つと考えられるのでやはり利用したいです．
機械学習手法は基本的に数値情報しか用いることができないため，文字列の情報をどうにかして数値に変換する必要があります．
一般に，非数値的な変数のことを**質的変数**や**カテゴリカル変数**と呼びます．

一つの方法として，**文字列一つ一つに適当に数字（例えば0（or 1）から始まる整数値）を割り当てる**というのが考えられます．
例えば，"type"では"organic"を1，"conventional"を2に変換する，"region"では"LasVegas"を1，"West"を2，"Indiannapolis"を3…，といった方法です．
しかし，この方法は多くのケースでは適切ではありません．
まず，基本的に，多くの予測モデルにおいて**特徴の値は重要な要素**です．
例えば線形モデルは以下の式で与えられます：
$$y(\mathbf{x}; \mathbf{w}) = \sum_{j=1}^D x_jw_j.$$ 
$x_j$というのは$w_j$の係数になっていますから，$x_j$の絶対値が大きい時，$w_j$の予測結果に対する影響度は大きくなります．
しかし，**文字列に割り当てた数字の値そのものに（基本的には）意味はない**はずです．
"LasVegas"を2にして，"West"を1にしてもよいはずです．
けれども，予測モデルに与えた時，数字の大きさが考慮されてしまいますから，この方法は適切ではないでしょう（したがって，**与えられたデータが数字で表現されていても，数字の値そのものに意味がない場合はそのまま使うのは適切ではない**ということになります）．

文字列情報の数値的な情報への変換方法で最もメジャーな方法として**one-hotエンコーディング**があります．
one-hotエンコーディングでは，**一つのカテゴリカル変数はカテゴリー数の次元のベクトルに変換**されます．
変換されたベクトルは，一つの要素が一つのカテゴリに対応していて，対応する要素の値が1でそれ以外の要素の値が0であるようなベクトルです．

例えば，"region"変数が"LasVegas"，"West"，"Indianapolis"，"Houston"の4種類の値を取ると仮定します（実際はもっと多いですが）．
この時，これらの文字列はそれぞれ，以下のような4次元のベクトルに変換されます．
- "LasVegas" -> $(1, 0, 0, 0)$
- "West" -> $(0, 1, 0, 0)$
- "Indianapolis" -> $(0, 0, 1, 0)$
- "Houston" -> $(0, 0, 0, 1)$

基本的に要素の対応関係は自由です（"West"と"Indianapolis"を入れ替えても良い）．
質的変数を含むデータでは，全ての質的変数にこのような変換を施して，数値的（量的）なベクトルと全ての質的変数のone-hotベクトルを連結したベクトルを特徴ベクトルとして用いることが多いです．

さて，この変換は適切なのでしょうか？
質的変数にも色々ありますから，全ての質的変数をダミー変数にすることが適切であるとは限りませんが，少なくとも雑に整数値を割り当てるよりはずっと良いです．
上の例の4次元ベクトルに対する線形モデルを例に考えてみます．
線形モデルの式は$y(\mathbf{x}; \mathbf{w}) = \sum_{j=1}^D x_jw_j$でした．
$\mathbf{x}$は今はどれか一つが$1$でそれ以外全てが$0$であるようなベクトルです．
$x_j=0$の時，$w_j$は使われませんから，$w_j$はj番目の場所におけるアボカドの値段を表しています．
$w_1$は"LasVegas"における値段の予測，$w_2$は"West"における値段の予測…といったようになり，雑に整数値を割り当てるよりはずっと良さそうですね．

one-hotエンコーディングでは，それぞれのカテゴリに一つの要素を割り当てて，完全に異なる特徴として扱います．
そのため，"Date"の例ですと，「日付が近い場合，値段の傾向も近い」と考えるのが自然かと思いますが，日付が異なれば完全に別のものとして扱うため，そのような傾向を陽に扱うことが難しくなります．
ですが，今回はとりあえず全ての文字列情報をone-hotエンコードします．

## pandasによるone-hotエンコーディング
それでは実際に"Date"，"type"，"region"の文字列情報をone-hotエンコーディングしてみましょう！
一見すると「書けるけど確実にそこそこ面倒だな」と思うかもしれません．
Pythonには辞書型という使いやすいハッシュテーブルが提供されていますが，それでも少し面倒かもしれません．
幸運なことに，pandasには`get_dummies`という非常に便利な関数が用意されています．
`get_dummies`を使うと，`DataFrame`の特定の列を簡単にone-hotエンコーディングできます．

ここまで退屈だったと思いますので，今回のQuizです．

### Quiz 1

[get_dummiesのドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) を読んで，以下のセルを埋め，訓練データとテストデータの"Date"，"type"，"region"をone-hotエンコードしてください．
**数値情報はone-hotエンコードせず，そのままにすること**．

この時，訓練データとテストデータを別々にエンコードすると，訓練データとテストデータが異なるエンコードをされてしまう可能性があります（今回のデータに対して`get_dummies`を使った場合は起こりませんが）．
そこで，**訓練データとテストデータを一旦連結**し，**連結したものをエンコード**し，**エンコード後に再び分ける**ことにします．
以下のセルでは最初に訓練データとテストデータを連結させた`d_train_test`を作っています（上が`d_train`，下が`d_test`）．
また，`columns_cat`はカテゴリカル変数の列名を集めたリストです．
これらを利用すると楽でしょう（使わなくともよいですが）．
訓練データとテストデータを分離させる方法については，前回の資料が参考になります．

変換後の行列が**231次元になっていれば正しいです**．

In [ ]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["Date", "type", "region"] # カテゴリカル変数の列名

d_train_test_onehot =　# ここを埋める．get_dummiesを使ってone-hotエンコーディング
d_train_onehot = # ここを埋める．d_train_test_onehotの訓練データ部分
d_test_onehot =  # ここを埋める．d_train_test_onehotのテストデータ部分
X_train_onehot = d_train_onehot.values # np.arrayに変換
X_test_onehot = d_test_onehot.values  # np.arrayに変換

`d_train_onehot`を`print`してみましょう．
数値情報を全てそのまま使いつつ，列の数が増えていることが分かります．
特に，`region_SouthCentral`のように`元のカテゴリ変数の名前_カテゴリ名`という名前の列名が増えていますね．
名前から明らかですが，`C_c`という名前の列があったとすると，それは`C`というカテゴリカル変数の`c`という名前のカテゴリに対応しています．

In [12]:
print(d_train_onehot)

       Unnamed: 0        4046        4225       4770  Small Bags  Large Bags  \
0              32  1134719.15   643464.88   95527.46   871562.92   104146.00   
1              43  1873878.11  2020327.87  302210.56  2091747.51   282242.79   
2              15      181.23      861.34       0.00      312.22     1020.40   
3              10   628591.78   284087.44   15249.83   200603.91   221593.36   
4              14        1.21      105.02       0.00    16555.36        0.00   
...           ...         ...         ...        ...         ...         ...   
12769          47     1565.49     4210.72       0.00     1133.33      756.16   
12770          46     7720.21    22562.00      37.04    38739.27    11885.39   
12771          43      386.65     5898.07       0.00      516.67      372.96   
12772           9      441.78     1817.74      15.61       72.11      907.40   
12773          28  3239258.39  2416002.43  112965.22   776961.13   214729.56   

       XLarge Bags  Date_2015-01-04  Da

では，新たに作った特徴ベクトルの行列を用いて線形回帰を学習し，結果を保存してみます．

In [13]:
lr.fit(X_train_onehot, y_train) # 学習
y_pred_test_lr_onehot = lr.predict(X_test_onehot)
print(y_pred_test_lr_onehot)

[1.45846179 1.09892611 1.36405931 ... 1.50938    1.97158584 1.20660231]


ついでに，バイアス項なしの線形回帰も学習し，その結果を保存しておきましょう．

In [14]:
lr_without_bias.fit(X_train_onehot, y_train) # 学習
y_pred_test_lr_onehot_without_bias = lr_without_bias.predict(X_test_onehot)
print(y_pred_test_lr_onehot_without_bias)

[1.45846171 1.09892608 1.36405934 ... 1.50938008 1.97158574 1.20660234]


数値的な情報のみを用いた場合と比べ，結果がかなり変わっていますね！

## 検証用データを用いたモデルの評価
### 過学習
さて，これまでの手順でひとまず２つのモデルと２つの特徴表現が得られ，合計4つの予測結果を作成しました．
`LinearRegression`のドキュメントを見れば分かりますが，他にも`normalize`というユーザが設定できる項目があります．
これを`True`と`False`の2通りで試すと，もうそれだけで8種類の予測結果になってしまいます．
しかし，一日の投稿回数が5回に制限されているため，全てのパターンを一日に投稿することはできません．
他にも多数の予測手法がありますし，今後，**試行錯誤する要素はどんどん増加するため，毎回の予測結果を全て投稿して評価することは現実的ではありません**．
そこで，**投稿せずに・投稿する前に**モデルを評価することを考えます．
投稿する前にモデルの評価をして，悪そうなモデルの予測結果は投稿せず，良さそうな場合だけ投稿する，というのは妥当な戦略でしょう．
また，コンペに限らず，モデルの事前の評価は機械学習手法の運用において常に非常に重要です．
機械学習を用いたサービスを考えた時に，とりあえず作ってみた予測モデルを本番環境で動かしてみるのは恐ろしいことでしょう．
本番環境で動かす前に一度評価して，良さそうであれば本番環境で動かすべきであるはずです．

さて，それではどのように投稿せずに・投稿する前にモデルを評価すれば良いのでしょうか？
コンペのスコアは**予測と正解**を用いて計算されます（今回は平均二乗誤差）．
残念ながら，テストデータの正解はわかっていないので，実際にコンペサイトから返ってくるスコアと同じものを事前に計算することはできません．
そこで，正解がわかっているデータ，すなわち訓練データについて予測を行って，誤差を計算して性能を見積もる，というのが考えられます．
しかし，この方法には問題があります．
モデルは**訓練データの誤差を小さくするように学習**しています．
非常に複雑なモデルを用いた時，訓練データに対する誤差をとにかく小さくしようとして（複雑なため，そのようなことが可能），その結果，訓練データに対して非常に精度の良い予測を行うが，訓練データに含まれないデータに対しては精度の低い予測を行ってしまう，ということがあります．
このような現象・状態を**過学習・過適合（overfitting）**と言います．データサイエンスの初回の講義で登場した**次数の大きい多項式回帰**が過学習の良い例です．
過学習するモデルというのは驚くほど簡単に作れてしまうため，**訓練データに対する誤差を用いてモデルを評価することは不適切です**．

例えば，以下のセルを動かしてみましょう（少し時間を要するかもしれません）．
以下のセルでは，過学習させるようにハイパーパラメータを選んだ**カーネルリッジ回帰**というモデルを動かしています（どのようなモデルかは今は気にしなくて良いです）．


In [15]:
from sklearn.kernel_ridge import KernelRidge # カーネルリッジ回帰を使えるようにする
kr = KernelRidge(kernel="rbf", gamma=1.0, alpha=0.0001)
kr.fit(X_train_num, y_train)
y_pred_train_kr = kr.predict(X_train_num) # 訓練データに対して予測
y_pred_test_kr = kr.predict(X_test_num) # テストデータに対して予測
np.savetxt(X=y_pred_test_kr, fname="y_pred_kr.txt") # テストデータの結果を保存

では，カーネルリッジ回帰の訓練データに対する予測`y_pred_train_kr`と訓練データの目標値`y_train`の間で平均二乗誤差を計算してみます．
自分で実装しても良いですが，sklearnに`mean_squared_error`という名前で既に実装されています．
以下のようにimportして使います．
詳しくは[ドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)を読んでみてください．

In [16]:
from sklearn.metrics import mean_squared_error
mse_train_kr = mean_squared_error(y_train, y_pred_train_kr) # 最初の引数に目標値（正解），次に予測を入れるとスコアが返ってくる
print(f"カーネルリッジ回帰の訓練誤差：{mse_train_kr}")

カーネルリッジ回帰の訓練誤差：2.1300517972845773e-08


e-08は10のマイナス8乗を意味していますから，これはもう**訓練データに対してはほとんど完全に正しく予測している**と言って良いでしょう．しかし，最後に"y_pred_kr.txt"としてファイル出力したテストデータに対する予測結果を実際に投稿してみると，**2.0前後の非常に悪いスコア（単にスコアの比を見ると，訓練データのおよそ100000000倍悪い！）**となっているはずです（信じがたい方はぜひ投稿してみてください）．

繰り返しになりますが，この例からもわかるように，学習に用いたデータで評価することは適切ではありません．

### 検証データの作成と検証データを用いた評価

学習に使ったデータでモデルを評価するのは不適切でした．
そこで，正解のわかっているデータの一部を**学習には用いず検証（評価）に用いる**ということを行います．
このようなデータを訓練データ（訓練集合）に対して**検証データ（検証集合，validation data, validation set)**と言います．

では実際にデータの分割をしてみましょう．
まず，数値情報だけを用いた`X_num`について行ってみます．
とりあえずここでは，ラベル付きデータ全体のおよそ**8割を訓練データ，残りを検証用**とします．
特に，現在のラベル付きデータの前半8割を訓練データ，後ろの2割を検証データとします．
先程のQuiz 1では`DataFrame`の分割を行いましたが，`np.array`の分割も同様に行うことができます．

In [17]:
n_valid= int(0.2*n_train) # 検証データの数．ラベル付きデータ数n_trainの2割を検証データにする
n_train = n_train - n_valid # 訓練データ数を計算し直す．検証データ以外全てなので，全体（n_train)からn_validationを引く 
X_valid_num = X_train_num[n_train:] # 現在のX_train_numの後半2割 = 前半8割以降
X_train_num = X_train_num[:n_train] # 現在のX_train_numの前半8割．再代入する（変数名を使い回す）

# 目標値も同じように分割する
y_valid = y_train[n_train:]
y_train = y_train[:n_train]

では，バイアス項ありの線形回帰，なしの線形回帰，さきほどのカーネルリッジ回帰の3種類のモデルを，分割した訓練データで学習し，検証データで評価してみましょう．

In [18]:
lr.fit(X_train_num, y_train)
y_pred_valid_lr = lr.predict(X_valid_num)
mse_valid_lr = mean_squared_error(y_valid, y_pred_valid_lr)
print(f"数値情報・バイアス項ありの線形回帰：{mse_valid_lr}")

lr_without_bias.fit(X_train_num, y_train)
y_pred_valid_lr_without_bias = lr_without_bias.predict(X_valid_num)
mse_valid_lr_without_bias = mean_squared_error(y_valid, y_pred_valid_lr_without_bias)
print(f"数値情報・バイアス項なしの線形回帰：{mse_valid_lr_without_bias}")

# カーネルリッジ回帰の学習・評価
kr.fit(X_train_num, y_train)
y_pred_valid_kr = kr.predict(X_valid_num)
mse_valid_kr = mean_squared_error(y_valid, y_pred_valid_kr)
print(f"数値情報・カーネルリッジ回帰：{mse_valid_kr}")

数値情報・バイアス項ありの線形回帰：0.15399482337165038
数値情報・バイアス項なしの線形回帰：2.006533691725563
数値情報・カーネルリッジ回帰：2.090286883320282


結果が出てきました．
カーネルリッジ回帰は訓練データに対する予測性能は良かったですが，訓練に使っていない検証データに対する予測性能が非常に悪い，教科書に載せたいような過学習の例となっていますね．
また，バイアス項を使わない線形回帰に対しては，予想通り低いスコアとなっています．

同様に，今回作ったone-hotベクトルに対しても訓練・検証の分割を行ってみます．

In [19]:
X_valid_onehot = X_train_onehot[n_train:] # 現在のX_train_onehotの後半2割 = 前半8割以降
X_train_onehot = X_train_onehot[:n_train] # 現在のX_train_onehotの前半8割．再代入する

同じように学習と評価を行います．

In [20]:
lr.fit(X_train_onehot, y_train)
y_pred_valid_lr_onehot = lr.predict(X_valid_onehot)
mse_valid_lr_onehot = mean_squared_error(y_valid, y_pred_valid_lr_onehot)
print(f"数値情報+one-hot・バイアス項ありの線形回帰：{mse_valid_lr_onehot}")

lr_without_bias.fit(X_train_onehot, y_train)
y_pred_valid_lr_without_bias_onehot = lr_without_bias.predict(X_valid_onehot)
mse_valid_lr_without_bias_onehot = mean_squared_error(y_valid, y_pred_valid_lr_without_bias_onehot)
print(f"数値情報+one-hot・バイアス項なしの線形回帰：{mse_valid_lr_without_bias_onehot}")

# カーネルリッジ回帰の学習・評価
kr.fit(X_train_onehot, y_train)
y_pred_valid_kr_onehot = kr.predict(X_valid_onehot)
mse_valid_kr_onehot = mean_squared_error(y_valid, y_pred_valid_kr_onehot)
print(f"数値情報+one-hot・カーネルリッジ回帰：{mse_valid_kr_onehot}")

数値情報+one-hot・バイアス項ありの線形回帰：0.04803772040301657
数値情報+one-hot・バイアス項なしの線形回帰：0.04817809697869687
数値情報+one-hot・カーネルリッジ回帰：2.090286883320282


線形回帰は非常に良くなっていますね！
また，one-hotベクトルも用いた場合，バイアス項の有無でほとんど差がありません．
one-hotベクトルがどのようなものかを考えるとなんとなく理由がわかるかもしれません．

一方，カーネルリッジ回帰は相変わらずひどいスコアです．
ただ，これはカーネルリッジ回帰が悪いのではなく，カーネルリッジ回帰の**使い方**が悪いです．
今後の資料では，使うのがやや難しいモデルをちゃんと使う，ということも行う予定です（線形回帰・カーネルリッジ回帰以外にも手法は色々あるので，ぜひ自身で調べて使ってみてください）．

この結果を見て，ようやく安心して**one-hotエンコーディングが上手く働いている**と言えます．
この線形回帰でテストデータの予測を計算して，自信を持って投稿してみましょう．

In [21]:
y_pred_test_lr_onehot = lr.predict(X_test_onehot)
np.savetxt(X=y_pred_test_lr_onehot, fname='y_pred_lr_onehot.txt')

今回はとりあえず前半8割を訓練，後半2割を検証としました．
しかし，分割の仕方によって結果も変わってしまいます．
分割・検証とハイパーパラメータの決定のもう少し賢い・便利な方法を次回行います．

## まとめ
 - 質的（カテゴリカル）データを変換する方法としてone-hotエンコーディングがある．一つ一つのカテゴリに特徴を割り当てて，そのカテゴリかそうでないかを0/1で表す．
 - 予測モデルを実際に使う前に（予測を提出する前に）モデルの評価をする必要がある．ラベルのある（もともと訓練用として渡されている）データを，訓練用のデータと検証用のデータに分割し，分割された訓練データだけを用いて学習し，学習に用いなかった検証用のデータを用いてモデルの評価をする．
 
 
量的変数と質的変数（カテゴリカルデータ）について，例えば「名義尺度」「順序尺度」「間隔尺度」等で調べてみると細かい分類やどのような操作が意味を持つのか（持たないのか）が出てくると思うので，興味がある方は調べてみると良いと思います．

## Answer

### Quiz 1
`pd.get_dummies`で`get_dummies`を呼び出すことができます．
最初の引数にone-hotエンコードを行う`DataFrame`を渡すので，`pd.get_dummies(d_train_test)`とすれば，とりあえずone-hotエンコードができます．
今，`columns_cat`に含まれる列だけone-hotエンコードをしたいです．
これは`columns`という名前の引数に`columns_cat`を指定することで可能になります．
したがって，`pd.get_dummies(d_train_test, columns=columns_cat)`とすればよいです．

次に，訓練とテストの取り出し方についてです
`DataFrame`において`[i:j]`とすることで，`i`番目から`j-1`番目までの行をを取り出すことができます．
また，`[:i]`とすることで`i-1`番目まで，`[i:]`とすることで`i`番目以降の行をを取り出すことができます．
したがって，`d_train_test[:n_train]`とすることで訓練データを，`d_train_test[n_train:]`とすることでテストデータを取り出せます．

In [10]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["Date", "type", "region"] # カテゴリカル変数の列名

d_train_test_onehot = pd.get_dummies(d_train_test, columns=columns_cat) # ここを埋める．get_dummiesを使ってone-hotエンコーディング
d_train_onehot = d_train_test_onehot[:n_train] # ここを埋める．d_train_test_onehotの訓練データ部分
d_test_onehot = d_train_test_onehot[n_train:] # ここを埋める．d_train_test_onehotのテストデータ部分
X_train_onehot = d_train_onehot.values # np.arrayに変換
X_test_onehot = d_test_onehot.values  # np.arrayに変換